In [1]:


import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

N_side = 16  # Choose your resolution
pixel_id = 42  # Example pixel index

# Get the boundaries of the pixel
boundaries = hp.boundaries(N_side, pixel_id, step=1)
theta, phi = hp.vec2ang(np.transpose(boundaries))

# Convert to degrees for plotting
theta_deg = np.degrees(theta)
phi_deg = np.degrees(phi)

# Plot
# plt.figure()
hp.visufunc.projscatter(theta, phi, lonlat=False, c='red')  # Plot edges
hp.visufunc.projtext(np.mean(theta), np.mean(phi), str(pixel_id), lonlat=False, color='blue')  # Center
plt.show()


<Figure size 640x480 with 0 Axes>

In [2]:
reddening_distance_slices = np.array([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])

In [3]:
def make_position_vector_from_spherical(l,b,r):
    x = r * np.cos(b) * np.cos(l)
    y = r * np.cos(b) * np.sin(l)
    z = r * np.sin(b)
    return np.array([x,y,z])



In [4]:
#### check that the ray is not in the plane defined by any of the healpix edges


In [5]:
class class_isrf():
    def __init__(self):
        

SyntaxError: incomplete input (279297834.py, line 3)

In [6]:
isrf_point_l = 0
isrf_point_b = 0
isrf_point_r = 0

star_l = 1
star_b = 1
star_r = 1


In [7]:


star_pos = make_position_vector_from_spherical(star_l,star_b,star_r)
isrf_pos = make_position_vector_from_spherical(isrf_point_l,isrf_point_b,isrf_point_r)

direction = isrf_pos - star_pos
direction_abs = np.dot(direction,direction)**0.5
star_dot_direction = np.dot(star_pos,direction)


In [8]:
def ray_linear_parameterization(star_pos,direction, t):
    return star_pos + t*direction


def p(star_pos,direction, t):
    x0,y0,z0 = star_pos
    a,b,c = direction
    p = ((x0 + t*a)**2 + (y0 + t*b)**2 + (z0 + t*c)**2)**0.5  
    return p

def find_crossing_positions(star_r,star_dot_direction,direction_abs,d):
    a = direction_abs**2
    b = 2*star_dot_direction
    c = star_r**2-d**2
    t1 = (-b + (b**2 - 4*a*c)**0.5)/(2*a)
    t2 = (-b - (b**2 - 4*a*c)**0.5)/(2*a)
    return t1,t2




#x = (-b +- (b^2-4ac)^0.5)/2a

In [9]:
find_crossing_positions(star_r,star_dot_direction,direction_abs,0.5)

(1.5, 0.5)

In [ ]:
x.shape

In [ ]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set the HEALPix resolution
nside = 10  # Adjust this for more or less detail
npix = hp.nside2npix(nside)

# Get the coordinates of the HEALPix pixels
theta, phi = hp.pix2ang(nside, np.arange(npix))

# Convert spherical coordinates to Cartesian coordinates for 3D plotting
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

# Create a 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for point visualization
ax.scatter(x, y, z, c='r', s=1)  # Color red, point size 1

# Enhance the plot
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Visualization of HEALPix Points')
ax.grid(True)

plt.show()


In [11]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
